# Equation of state (EOS)
For the source code, see [eos](https://workgraph-collections.readthedocs.io/en/latest/qe/module.html#workgraph_collections.ase.espresso.eos.eos_workgraph).

## Example: Silicon
### Prepare the inputs and visualize the WorkGraph

In [6]:
from aiida import load_profile, orm
from ase.build import bulk
from workgraph_collections.qe.eos import EosWorkgraph

load_profile()

structure = orm.StructureData(ase=bulk("Si"))
code = orm.load_code("pw-7.4@localhost")
parameters = {
    "CONTROL": {
        "calculation": "scf",
    },
    "SYSTEM": {
        "ecutwfc": 30,
        "ecutrho": 240,
        "occupations": "smearing",
        "smearing": "gaussian",
        "degauss": 0.1,
    },
}
# Load the pseudopotential family.
pseudo_family = orm.load_group("SSSP/1.3/PBEsol/efficiency")
pseudos = pseudo_family.get_pseudos(structure=structure)
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}
scf_inputs = {
    "pw": {
        "code": code,
        "parameters": orm.Dict(parameters),
        "pseudos": pseudos,
        "metadata": metadata,
    },
    "kpoints": kpoints,
}
# ===============================================================================
wg = EosWorkgraph.build(
    structure=structure,
    scales=[0.98, 0.99, 1.0, 1.01, 1.02],
    scf_inputs=scf_inputs,
)
wg.to_html()

### Run the workflow

In [4]:
wg.run()

10/08/2025 11:02:15 AM <2482962> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201574|WorkGraphEngine|continue_workgraph]: tasks ready to run: scale_structure
10/08/2025 11:02:16 AM <2482962> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201574|WorkGraphEngine|update_task_state]: Task: scale_structure, type: CALCFUNCTION, finished.
10/08/2025 11:02:16 AM <2482962> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201574|WorkGraphEngine|continue_workgraph]: tasks ready to run: all_scf
10/08/2025 11:02:16 AM <2482962> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201574|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 201582
10/08/2025 11:02:17 AM <2482962> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201582|WorkGraphEngine|continue_workgraph]: tasks ready to run: PwBaseWorkChain,PwBaseWorkChain1,PwBaseWorkChain2,PwBaseWorkChain3,PwBaseWorkChain4
10/08/2025 11:

{'result': <Dict: uuid: a29908bc-8353-4ea4-85ac-6c9909e148d3 (pk: 201647)>}

## Print the results

In [5]:
#------------------------- Print the output -------------------------
data = wg.outputs.result.value.get_dict()
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**data))



Result: 
B: 83.918906760674
v0: 40.947419077162
e0: -308.18973202602
v0: 40.947419077162
